In [56]:
import tensorflow as tf
import numpy as np
from functools import reduce
from numpy import unique, array, vectorize
from sklearn.metrics import accuracy_score, f1_score

class SVMClassifier:

    def __init__(self, train_data=None):
        data, labels = train_data

        labels = self._transform_labels(labels)
        
        data = self._flatten_input(data)
        
        self.train_data = (data, labels)

        self._open_session()

        self.assemble_graph()

        if train_data:
            self.train()     

    def assemble_graph(self, learning_rate = 0.02):
        self.X = tf.placeholder(name="input", dtype=tf.float32, shape=(None, self.train_data[0].shape[1]))
        self.y = tf.placeholder(name="label", dtype=tf.float32, shape=(None,1))
        self.W = tf.Variable(tf.zeros(shape=[self.train_data[0].shape[1], 1],
                                     dtype = tf.float32))
        self.b = tf.Variable(tf.zeros(shape=[1], dtype = tf.float32))
        self.pred = tf.subtract(tf.matmul(self.X,self.W),self.b)
        self.max = tf.maximum(0.0,tf.subtract(1.0,tf.multiply(self.y,self.pred)))
        self.loss = tf.reduce_mean(self.max)
        self.opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
    def train(self, epochs=20, minibatch_size=256):
        batch = self._create_minibatches(minibatch_size)
        writer = tf.summary.FileWriter('./tmp', tf.get_default_graph())
        for i in range(epochs):
            loss = 0
            for j in batch:
                feed_dict = {self.X: j[0], self.y: np.array([j[1]]).T}
                _, epoch_loss = self.sess.run([self.opt, self.loss], feed_dict = feed_dict)
                loss += epoch_loss
            print(f"Epoch {i}: {loss/len(batch)}")
        writer.close()
        
    def predict(self, data):
        data = self._flatten_input(data)
        feed_dict = {self.X:data}
        prediction = self.sess.run(self.pred, feed_dict = feed_dict)
        prediction = np.sign(prediction)
        prediction[prediction == -1] = 0
        return prediction
        
    def _create_minibatches(self, minibatch_size):
        pos = 0

        data, labels = self.train_data
        n_samples = len(labels)

        batches = []
        while pos + minibatch_size < n_samples:
            batches.append((data[pos:pos+minibatch_size,:], labels[pos:pos+minibatch_size]))
            pos += minibatch_size

        if pos < n_samples:
            batches.append((data[pos:n_samples,:], labels[pos:n_samples]))

        return batches

    def _transform_labels(self, labels):
        transformed = [-1 if i == 0 else 1 for i in labels]
        transformed = np.array(transformed)
        return transformed

    def _flatten_input(self, data):
        return np.reshape(data,newshape=[-1,784])

    def _open_session(self):
        self.sess = tf.Session()


if __name__ == "__main__":



    def mnist_to_binary(train_data, train_label, test_data, test_label):

        binarized_labels = []
        for labels in [train_label, test_label]:
            remainder_2 = vectorize(lambda x: x%2)
            binarized_labels.append(remainder_2(labels))

        train_label, test_label = binarized_labels

        return train_data, train_label, test_data, test_label




    ((train_data, train_labels),
        (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

    train_data, train_labels, test_data, test_labels = mnist_to_binary(train_data, train_labels, eval_data, eval_labels)

    svm = SVMClassifier((train_data, train_labels))
    print("Testing score f1: {}".format(f1_score(test_labels, svm.predict(test_data))))


Epoch 0: 767.0387573891497
Epoch 1: 654.7200182975607
Epoch 2: 593.8478164672852
Epoch 3: 555.564276837288
Epoch 4: 567.7827603279276
Epoch 5: 539.376040324759
Epoch 6: 515.8916279569585
Epoch 7: 544.698980420701
Epoch 8: 546.2159795233543
Epoch 9: 496.74433015863946
Epoch 10: 520.8186554279733
Epoch 11: 546.86419015438
Epoch 12: 477.54597486130734
Epoch 13: 558.9010512981009
Epoch 14: 484.2106445637155
Epoch 15: 494.7110846336852
Epoch 16: 504.16928694704745
Epoch 17: 529.6609443177568
Epoch 18: 478.3484826595225
Epoch 19: 498.0356779707239
Testing score f1: 0.8929463770901402


### CNN

In [4]:
import tensorflow as tf
import numpy as np

import tensorflow as tf
import numpy as np
from functools import reduce
from numpy import unique, array, vectorize
from sklearn.metrics import accuracy_score, f1_score

class CNNClassifier:

    def __init__(self, train_data=None):
        data, labels = train_data
        
        self.train_data = (data, labels)

        self._open_session()

        self.assemble_graph()

        if train_data:
            self.train()  


    def assemble_graph(self, learning_rate = 0.02):
        self.X = tf.placeholder(name='input', dtype = tf.float32, shape = [None, self.train_data[0].shape[1], self.train_data[0].shape[2], 1])
        self.y = tf.placeholder(name='output',dtype = tf.float32, shape = [None])
        conv1 = tf.layers.conv2d(inputs = self.X,filters = 3,
                                 kernel_size = [5, 5],padding="same",activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size=[2, 2], strides=2)
        conv2 = tf.layers.conv2d(inputs = pool1, filters = 3,
                                kernel_size = [5, 5], padding = "same", activation = tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2], strides = 2)
        pool2_reshape = tf.reshape(pool2, [-1, 7*7*3])
        self.dense = tf.layers.dense(inputs = pool2_reshape, units = 4, activation = tf.nn.relu)
        dense_logit1 = tf.layers.dense(inputs = self.dense, units = 1)
        self.dense_logit2 = tf.nn.sigmoid(dense_logit1)
        loss_ = tf.nn.sigmoid_cross_entropy_with_logits(labels = self.y, logits = tf.reshape(self.dense_logit2, [-1]))
        self.loss = tf.reduce_mean(loss_)
        self.opt = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
    def train(self, epochs = 10, minibatch_size = 256):
        batch = self._create_minibatches(minibatch_size)
        writer = tf.summary.FileWriter('./tmp', tf.get_default_graph())
        for i in range(epochs):
            loss = 0
            for j in batch:
                feed_dict = {self.X: j[0][:,:,:,np.newaxis], self.y: np.array(j[1])}
                _, epoch_loss = self.sess.run([self.opt, self.loss], feed_dict = feed_dict)
                loss += epoch_loss
            print(f"Epoch {i}: {loss/len(batch)}")
        writer.close()
        
    def predict(self, data):
        feed_dict = {self.X: data[:,:,:,np.newaxis]}
        prediction = self.sess.run(self.dense_logit2, feed_dict = feed_dict)
        predictions = [int(x) for x in prediction]
        return predictions
        
    def _create_minibatches(self, minibatch_size):
        pos = 0

        data, labels = self.train_data
        n_samples = len(labels)

        batches = []
        while pos + minibatch_size < n_samples:
            batches.append((data[pos:pos+minibatch_size,:], labels[pos:pos+minibatch_size]))
            pos += minibatch_size

        if pos < n_samples:
            batches.append((data[pos:n_samples,:], labels[pos:n_samples]))
        return batches
    
    def _open_session(self):
        self.sess = tf.Session()

def mnist_to_binary(train_data, train_label, test_data, test_label):

    binarized_labels = []
    for labels in [train_label, test_label]:
        remainder_2 = vectorize(lambda x: x%2)
        binarized_labels.append(remainder_2(labels))

    train_label, test_label = binarized_labels

    return train_data, train_label, test_data, test_label

((train_data, train_labels),
        (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data, train_labels, test_data, test_labels = mnist_to_binary(train_data, train_labels, eval_data, eval_labels)

cnn = CNNClassifier((train_data, train_labels))
print("Testing score f1: {}".format(f1_score(test_labels, cnn.predict(test_data))))



Epoch 0: 0.6115232728897257
Epoch 1: 0.5695848541056856
Epoch 2: 0.5544047918725521
Epoch 3: 0.5427426272250236
Epoch 4: 0.5379628802867646
Epoch 5: 0.5336445318891647
Epoch 6: 0.5315862609985027
Epoch 7: 0.5284078258149167
Epoch 8: 0.5259947237816263
Epoch 9: 0.5247536316831061
Testing score f1: 0.9274546591619761
